In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

In [4]:
vggish_model_url = 'https://storage.googleapis.com/audioset/vggish_model.ckpt'
vggish_pca_params_url = 'https://storage.googleapis.com/audioset/vggish_pca_params.npz'

vggish_model = tf.keras.applications.vggish.VGGish(input_shape=(96, 64, 1), include_top=False)
vggish_model.load_weights(tf.keras.utils.get_file(
    'vggish_model_weights_tf2.h5',
    vggish_model_url,
    cache_subdir='models',
    file_hash='5f123b940457303214615479366079c691d45e18b74215e79bc7b369b3d21659'
))
vggish_model.trainable = False

# VGGish'in çıkışlarını özelliklerin özetini çıkarmak için kullan
x = Flatten()(vggish_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)  # num_classes, tahmin edilecek duyguların sayısı


AttributeError: module 'keras.applications' has no attribute 'vggish'

In [ ]:
model = Model(inputs=vggish_model.input, outputs=output)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
data_path = 'D:/yapayzekafinal/ses'

In [ ]:
data = []
labels = []
emotions = {'angry_05': 0, 'boredom': 1, 'calm_02': 2, 'disgust_07': 3, 'fearful_06': 4, 'happy_03': 5, 'neutral_01': 6, 'pain': 7, 'sad_04': 8, 'suprised_08': 9}

for emotion in emotions.keys():
    emotion_path = os.path.join(data_path, emotion)
    for filename in os.listdir(emotion_path):
        if filename.endswith('.wav'):
            filepath = os.path.join(emotion_path, filename)
            feature = extract_feature(filepath)  # extract_feature fonksiyonunu çağırarak ses özelliklerini çıkar
            data.append(feature)
            labels.append(emotions[emotion])


In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

In [ ]:
model.fit(x_train, y_train_encoded, batch_size=64, epochs=50, validation_data=(x_test, y_test_encoded))